In [5]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

# ## DEFINE SENSITIVE VARIABLES
# NESSIE_URI = "http://nessie:19120/api/v1"
# MINIO_ACCESS_KEY = "admin"
# MINIO_SECRET_KEY = "password"



conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
#         .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
#   		#SQL Extensions
#         .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
#   		#Configuring Catalog
#         .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
#         .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
#         .set('spark.sql.catalog.nessie.ref', 'main')
#         .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
#         .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
#         .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
#         .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
#         .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
#   		#MINIO CREDENTIALS
#         .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
#         .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open")

# ## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
# spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson3 USING iceberg PARTITIONED BY (countryOfOriginCode) AS SELECT * FROM csv_open_2023 ORDER BY countryOfOriginCode;").show()

# ## ALTER PARTITIONING BASED ON TRUNCATE FIRST LETTER OF firstName
# spark.sql("ALTER TABLE nessie.df_open_2023_lesson3 ADD PARTITION FIELD truncate(1, firstName)").show()

# ## REWRITE ALL THE DATA SO IT ALL USES UPDATED PARTITONING
# spark.sql("CALL nessie.system.rewrite_data_files('df_open_2023_lesson3')").show()

Spark Running


In [6]:
csv_df.show()

+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      469656|    Jeffrey Adler|  Jeffrey|      Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059| CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|
|      300638|   Tola Morakinyo|     Tola|  Morakinyo|   ACT|     M|                 US|    

In [7]:
spark.sql("CREATE TABLE IF NOT EXISTS df_open_lesson5 AS SELECT * FROM csv_open;").show()


25/03/04 12:25:57 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);
'CreateTable `default`.`df_open_lesson5`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Ignore
+- Project [competitorId#169, competitorName#170, firstName#171, lastName#172, status#173, gender#174, countryOfOriginCode#175, countryOfOriginName#176, regionId#177, regionName#178, affiliateId#179, affiliateName#180, age#181, height#182, weight#183, overallRank#184, overallScore#185, genderId#186, year#187]
   +- SubqueryAlias csv_open
      +- View (`csv_open`, [competitorId#169,competitorName#170,firstName#171,lastName#172,status#173,gender#174,countryOfOriginCode#175,countryOfOriginName#176,regionId#177,regionName#178,affiliateId#179,affiliateName#180,age#181,height#182,weight#183,overallRank#184,overallScore#185,genderId#186,year#187])
         +- Relation [competitorId#169,competitorName#170,firstName#171,lastName#172,status#173,gender#174,countryOfOriginCode#175,countryOfOriginName#176,regionId#177,reg